In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as stats
import matplotlib.pyplot as plt

In [ ]:
from growthstandards import rv_coords, GrowthStandards, XrCompoundRV
from growthstandards.bcs_ext.scipy_ext import BCCG, BCPE

In [ ]:
def coord_da(vs, name, attrs={}):
    return xr.DataArray(vs, dims=name).assign_attrs(attrs).assign_coords({name: lambda da: da})

In [ ]:
_vs = "weight", "len_hei", "bmi", "wfl", "wfh"

fig, axs = plt.subplots(len(_vs), 2, layout="constrained", figsize=(12, 5 * len(_vs)))

for raxs, v in zip(axs, _vs):
    grv = GrowthStandards[v]
    _y = coord_da(np.linspace(grv.ppf(0.01).min(), grv.ppf(0.99).max(), num=1000), v, getattr(grv, "attrs", {}))
    p2d = grv.pdf(_y, apply_kwargs=dict(keep_attrs="drop_conflicts"))
    for ax, s in zip(raxs, ("Female", "Male")):
        p2d.sel(sex=s).drop("sex_enum").plot.imshow(y=v, add_colorbar=False, ax=ax)

In [ ]:
def _rtol(a, b):
    return abs(a - b) / abs(a)

In [ ]:
from growthstandards import rv_to_ds
# grv = GrowthStandards["len_hei"]
grv = GrowthStandards["length"]
gds = rv_to_ds(grv)
print(gds["mu"].values)
print(grv.median().values)
print(np.allclose(gds["mu"], grv.median()))
print(gds["sigma"].values)
_grv_approx_sigma = (
    np.arcsinh(
        ((grv.ppf(0.75) - grv.ppf(0.25)) / grv.median()) / 2
    ) / stats.norm.ppf(0.75)
)
print(_grv_approx_sigma.values)
req_rtol = _rtol(gds["sigma"], _grv_approx_sigma).max()
print(f"Required relative tolerance: {req_rtol:e}")

In [ ]:
rv_to_ds(GrowthStandards["wfl"])["sigma"].plot.line(x="length")

In [ ]:
del rv_to_ds

In [ ]:
_q0 = 1e-4
quantiles = coord_da([_q0, 0.25, 0.5, 0.75, 1 - _q0], "quantile")
quantiles

### TODO: derive P(G_A) from P(W | L), P(W_A), P(L_A), where G = W / L, and A is indexed by age
$$
\begin{align}
A &= \text{age} \\
G &= W / L \\
p_G(g) &= \int_L |l| p_{W,L}(g l, l) \mathrm{d}l \\
&= \int_L |l| p_{W | L}(g l | l) p_L(l) \mathrm{d}l \\
&= \mathrm{E}_L[|L| p_{W | L}(g L | L)] \\
\mathrm{P}(G=g) &= \int_L |l| \mathrm{P}(W = g l | L = l) \mathrm{P}(L = l) \mathrm{d}l
\end{align}
$$

In [ ]:
len_rv = GrowthStandards["length"]
hei_rv = GrowthStandards["height"]
gfl_rv = GrowthStandards["gfl"]
gfh_rv = GrowthStandards["gfh"]

# length_da = rv_coords(gfl_rv)["length"]
# height_da = rv_coords(gfh_rv)["height"]

growth_da = coord_da(np.linspace(0.01, 0.25, 100), "growth")
# growth_da = coord_da(np.linspace(0.01, 0.25, 1_000), "growth")

# p_length_da = len_rv.pdf(length_da)
# p_height_da = hei_rv.pdf(height_da)
# _p_lg_l = gfl_rv.pdf(growth_da)
# _p_hg_h = gfh_rv.pdf(growth_da)
# p_g_l = (_p_lg_l * p_length_da).integrate("length")
# p_g_h = (_p_hg_h * p_height_da).integrate("height")
# p_g = xr.combine_by_coords([p_g_l, p_g_h])

growth_len_rv = XrCompoundRV(gfl_rv, len_rv, "length")
growth_hei_rv = XrCompoundRV(gfh_rv, hei_rv, "height")

p_g = xr.combine_by_coords([
    growth_len_rv.pdf(growth_da),
    growth_hei_rv.pdf(growth_da),
])
p_g

In [ ]:
p_g.integrate("growth")

In [ ]:
# _cdf_lg_l = gfl_rv.cdf(growth_da)
# _cdf_hg_h = gfh_rv.cdf(growth_da)
# cdf_g_l = (_cdf_lg_l * p_length_da).integrate("length")
# cdf_g_h = (_cdf_hg_h * p_height_da).integrate("height")
# cdf_g = xr.combine_by_coords([cdf_g_l, cdf_g_h])

cdf_g = xr.combine_by_coords([
    growth_len_rv.cdf(growth_da),
    growth_hei_rv.cdf(growth_da),
])
cdf_g

In [ ]:
distr_qs = xr.combine_by_coords([
    growth_len_rv.ppf(quantiles),
    growth_hei_rv.ppf(quantiles),
])
distr_qs

In [ ]:
fig, axs = plt.subplots(3, 2, layout="constrained", figsize=(14, 15))
p_g.isel(sex=0).plot.imshow(x="age", ax=axs[0, 0])
p_g.isel(sex=1).plot.imshow(x="age", ax=axs[0, 1])

growth_cdf = cdf_g
growth_cdf.isel(sex=0).plot.imshow(x="age", ax=axs[1, 0])
growth_cdf.isel(sex=1).plot.imshow(x="age", ax=axs[1, 1])

growth_cdf = cdf_g - p_g.cumulative_integrate("growth")
growth_cdf.isel(sex=0).plot.imshow(x="age", ax=axs[2, 0])
growth_cdf.isel(sex=1).plot.imshow(x="age", ax=axs[2, 1])
del growth_cdf

In [ ]:
p_g.isel(age=0).plot(x="growth", hue="sex")

In [ ]:
_p_g_x = p_g.growth.values
_dp_g_x = _p_g_x[1] - _p_g_x[0]
_p_g_bins = np.append(_p_g_x - _dp_g_x / 2, _p_g_x[-1] + _dp_g_x / 2)
_distr = stats.rv_histogram((p_g.isel(age=0, sex=0), _p_g_bins), density=True)
_distr

In [ ]:
_distr_median = xr.apply_ufunc(
    lambda p: stats.rv_histogram((p, _p_g_bins), density=True).median(),
    p_g,
    input_core_dims=[["growth"]],
    vectorize=True,
)
_distr_median

In [ ]:
distr_median = distr_qs.sel(quantile=0.5)
distr_median

In [ ]:
_q0 = 1e-4
_q = quantiles
_distr_qs = xr.apply_ufunc(
    lambda p, q: stats.rv_histogram((p, _p_g_bins), density=True).ppf(q),
    p_g,
    _q,
    input_core_dims=[["growth"], _q.dims],
    output_core_dims=[_q.dims],
    vectorize=True,
)
_distr_qs

In [ ]:
_distr_median.plot.line(x="age", hue="sex")
distr_median.plot.line(x="age", hue="sex")

In [ ]:
_distr_qcv = (3/4) * (_distr_qs.sel(quantile=0.75) - _distr_qs.sel(quantile=0.25)) / _distr_median
_distr_qcv

In [ ]:
distr_qcv = (3/4) * (distr_qs.sel(quantile=0.75) - distr_qs.sel(quantile=0.25)) / distr_median
distr_qcv

In [ ]:
_distr_qcv.plot.line(x="age", hue="sex")

In [ ]:
distr_qcv.plot.line(x="age", hue="sex")

In [ ]:
_approx_sigma = np.arcsinh(_distr_qcv / 1.5) / stats.norm.ppf(0.75)
_approx_sigma

In [ ]:
approx_sigma = np.arcsinh(distr_qcv / 1.5) / stats.norm.ppf(0.75)
approx_sigma

In [ ]:
_approx_sigma.plot.line(x="age", hue="sex")
approx_sigma.plot.line(x="age", hue="sex")

In [ ]:
_distr_mean, _distr_var, _distr_skew, _distr_kurt = xr.apply_ufunc(
    lambda p: stats.rv_histogram((p, _p_g_bins), density=True).stats(moments="mvsk"),
    p_g,
    input_core_dims=[["growth"]],
    output_core_dims=[(), (), (), ()],
    vectorize=True,
)
xr.Dataset(dict(zip("mvsk", (_distr_mean, _distr_var, _distr_skew, _distr_kurt))))

In [ ]:
_distr_mean.plot.line(x="age", hue="sex")
_distr_median.plot.line(x="age", hue="sex")

In [ ]:
_distr_var.plot.line(x="age", hue="sex")

In [ ]:
(np.sqrt(_distr_var) / _distr_median).plot.line(x="age", hue="sex")

In [ ]:
(np.sqrt(_distr_var) / _distr_mean).plot.line(x="age", hue="sex")

In [ ]:
# (np.sqrt(_distr_var) / _distr_median).plot.line(x="age", hue="sex")
(np.sqrt(_distr_var) / _distr_mean).plot.line(x="age", hue="sex")
_distr_qcv.plot.line(x="age", hue="sex")

In [ ]:
approx_sigma.plot.line(x="age", hue="sex")
(np.sqrt(_distr_var) / _distr_mean).plot.line(x="age", hue="sex")

In [ ]:
_distr_skew.plot.line(x="age", hue="sex")

In [ ]:
q1, q2, q3 = distr_qs.sel(quantile=0.25), distr_qs.sel(quantile=0.50), distr_qs.sel(quantile=0.75)
qSkew = ((q3 - q2) - (q2 - q1)) / (q3 - q1)
qSkew.plot.line(x="age", hue="sex")

In [ ]:
np.sqrt(qSkew).plot.line(x="age", hue="sex")

In [ ]:
(qSkew / _distr_skew).plot.line(x="age", hue="sex")

In [ ]:
_distr_kurt.plot.line(x="age", hue="sex")

In [ ]:
p_g.isel(age=0).plot.line(x="growth", hue="sex")

In [ ]:
p_g.isel(age=57).plot.line(x="growth", hue="sex")

In [ ]:
N_SAMPLE = 100
_distr_sample = xr.apply_ufunc(
    lambda p: stats.rv_histogram((p, _p_g_bins), density=True).rvs(size=N_SAMPLE),
    p_g,
    input_core_dims=[["growth"]],
    output_core_dims=[["sample"]],
    vectorize=True,
)
_distr_sample

In [ ]:
fig, axs = plt.subplots(2, 2, layout="constrained", figsize=(14, 12))
_distr_sample.isel(sex=0).plot.scatter(x="age", ax=axs[0, 0])
_distr_sample.isel(sex=1).plot.scatter(x="age", ax=axs[0, 1])

_trans_distr_sample = (_distr_sample / _distr_median) **(-1.511e-01)
_trans_distr_sample.isel(sex=0).plot.scatter(x="age", ax=axs[1, 0])
_trans_distr_sample.isel(sex=1).plot.scatter(x="age", ax=axs[1, 1])

In [ ]:
xr.Dataset({
    "median": _trans_distr_sample.median(dim="sample"),
    "std": _trans_distr_sample.std(dim="sample"),
    "skew": _trans_distr_sample.reduce(stats.skew, dim="sample"),
})

In [ ]:
import scipy.optimize as optimize

In [ ]:
_sex = "Male"

_m = _distr_median.sel(sex=_sex)
_s = np.sqrt(_distr_var).sel(sex=_sex)
# _cv = _s / _m
_cv = approx_sigma.sel(sex=_sex)
_truth = p_g.sel(sex=_sex)

_norm_cost = (
    0.5 * ((_truth - xr.apply_ufunc(stats.norm.pdf, p_g.growth, _m, _s)) ** 2).sum()
)
print("Normal Fit:")
print("        cost:", _norm_cost.values)
_bccg_fit = optimize.least_squares(
    lambda x: (
        (_truth - xr.apply_ufunc(BCCG.pdf, p_g.growth, _m, _cv, *x)) ** 2
    ).values.flatten(),
    x0=-0.3521,
)
print("BCCG Fit:")
print(_bccg_fit)
_bcpe_fit = optimize.least_squares(
    lambda x: (
        (_truth - xr.apply_ufunc(BCPE.pdf, p_g.growth, _m, _cv, *x)) ** 2
    ).values.flatten(),
    x0=[-0.3, 1],
)
print("BCPE Fit:")
print(_bcpe_fit)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 12))

((_truth - xr.apply_ufunc(stats.norm.pdf, p_g.growth, _m, _s)) ** 2).sum(
    dim="growth"
).plot.line(ax=axs[0], label="Norm cost")
ax.legend()
((_truth - xr.apply_ufunc(BCCG.pdf, p_g.growth, _m, _cv, *_bccg_fit.x)) ** 2).sum(
    dim="growth"
).plot.line(ax=axs[1], label="BCCG cost")
((_truth - xr.apply_ufunc(BCPE.pdf, p_g.growth, _m, _cv, *_bcpe_fit.x)) ** 2).sum(
    dim="growth"
).plot.line(ax=axs[1], label="BCPE cost")
ax.legend()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(18, 12), sharex=True, layout="constrained")
_idx = 57

ax = axs[0]
_truth.isel(age=_idx).plot.line(ax=ax, x="growth", label="truth")
xr.apply_ufunc(stats.norm.pdf, p_g.growth, _m, _s).isel(age=_idx).plot.line(
    ax=ax, x="growth", ls="--", label="norm fit"
)
ax.set_title("Normal Fit")
ax = axs[1]
_truth.isel(age=_idx).plot.line(ax=ax, x="growth", label="truth")
xr.apply_ufunc(BCCG.pdf, p_g.growth, _m, _cv, *_bccg_fit.x).isel(age=_idx).plot.line(
    ax=ax, x="growth", ls="--", label="bccg fit"
)
ax.set_title("BCCG Fit")
ax = axs[2]
_truth.isel(age=_idx).plot.line(ax=ax, x="growth", label="truth")
xr.apply_ufunc(BCPE.pdf, p_g.growth, _m, _cv, *_bcpe_fit.x).isel(age=_idx).plot.line(
    ax=ax, x="growth", ls="--", label="bcpe fit"
)
ax.set_title("BCPE Fit")
_q_lims = _distr_qs.sel(sex=_sex).isel(age=_idx)
ax.set_xlim(_q_lims.isel(quantile=0), _q_lims.isel(quantile=-1))

In [ ]:
_sex = "Male"

_m = _distr_median.sel(sex=_sex)
_s = np.sqrt(_distr_var).sel(sex=_sex)
# _cv = _s / _m
_cv = approx_sigma.sel(sex=_sex)
_truth = np.log(p_g.sel(sex=_sex))

_norm_cost = (
    0.5 * ((_truth - xr.apply_ufunc(stats.norm.logpdf, p_g.growth, _m, _s)) ** 2).sum()
)
print("Normal Fit:")
print("        cost:", _norm_cost.values)
_bccg_fit = optimize.least_squares(
    lambda x: (
        (_truth - xr.apply_ufunc(BCCG.logpdf, p_g.growth, _m, _cv, *x)) ** 2
    ).values.flatten(),
    x0=-0.3,
)
print("BCCG Fit:")
print(_bccg_fit)
_bcpe_fit = optimize.least_squares(
    lambda x: (
        (_truth - xr.apply_ufunc(BCPE.logpdf, p_g.growth, _m, _cv, *x)) ** 2
    ).values.flatten(),
    x0=[-0.3, 1],
)
print("BCPE Fit:")
print(_bcpe_fit)

_truth = p_g.sel(sex=_sex)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 12))

((_truth - xr.apply_ufunc(stats.norm.pdf, p_g.growth, _m, _s)) ** 2).sum(
    dim="growth"
).plot.line(ax=axs[0], label="Norm cost")
ax.legend()
((_truth - xr.apply_ufunc(BCCG.pdf, p_g.growth, _m, _cv, *_bccg_fit.x)) ** 2).sum(
    dim="growth"
).plot.line(ax=axs[1], label="BCCG cost")
((_truth - xr.apply_ufunc(BCPE.pdf, p_g.growth, _m, _cv, *_bcpe_fit.x)) ** 2).sum(
    dim="growth"
).plot.line(ax=axs[1], label="BCPE cost")
ax.legend()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(18, 12), sharex=True, layout="constrained")
_idx = 100

ax = axs[0]
_truth.isel(age=_idx).plot.line(ax=ax, x="growth", label="truth")
xr.apply_ufunc(stats.norm.pdf, p_g.growth, _m, _s).isel(age=_idx).plot.line(
    ax=ax, x="growth", ls="--", label="norm fit"
)
ax.set_title("Normal Fit")
ax = axs[1]
_truth.isel(age=_idx).plot.line(ax=ax, x="growth", label="truth")
xr.apply_ufunc(BCCG.pdf, p_g.growth, _m, _cv, *_bccg_fit.x).isel(age=_idx).plot.line(
    ax=ax, x="growth", ls="--", label="bccg fit"
)
ax.set_title("BCCG Fit")
ax = axs[2]
_truth.isel(age=_idx).plot.line(ax=ax, x="growth", label="truth")
xr.apply_ufunc(BCPE.pdf, p_g.growth, _m, _cv, *_bcpe_fit.x).isel(age=_idx).plot.line(
    ax=ax, x="growth", ls="--", label="bcpe fit"
)
ax.set_title("BCPE Fit")
_q_lims = _distr_qs.sel(sex=_sex).isel(age=_idx)
ax.set_xlim(_q_lims.isel(quantile=0), _q_lims.isel(quantile=-1))